# Integrated Data Models

The models in this notebook are trained on all collected data, including historical price data, sentiment and public interest data, correlated asset data, and feature engineered technical indicators.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, Conv1D, MaxPooling1D, Flatten

warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

In [2]:
data=pd.read_csv(r'C:\Users\madha\Desktop\Dissertation\Data\Master Data\Feature Engineering\FeatureEngineered_XRP.csv')

In [3]:
data.tail()

,date,name,open,high,low,close,volume,marketCap,daily_weighted_sentiment,comment_volume,...,boll,boll_ub,boll_lb,boll_width,kdjk,kdjd,kdj_signal,macd,macds,macdh
345,2025-07-10,2781,2.405636,2.558056,2.397295,2.546683,6.049788e+09,1.504249e+11,-0.187919,48.0,...,2.220419,2.451427,1.989410,0.462017,84.705485,76.136926,0.0,0.052895,0.018265,0.034630
346,2025-07-11,2781,2.546631,2.964270,2.517547,2.734864,1.578910e+10,1.617161e+11,-0.256150,49.0,...,2.254284,2.568400,1.940168,0.628232,79.791463,77.355105,0.0,0.083217,0.031255,0.051961
347,2025-07-12,2781,2.734850,2.841254,2.669910,2.737636,8.647115e+09,1.618808e+11,0.024163,37.0,...,2.290465,2.651283,1.929646,0.721637,76.636424,77.115545,-1.0,0.106246,0.046254,0.059992
348,2025-07-13,2781,2.737636,2.878500,2.726897,2.835941,7.159804e+09,1.676928e+11,-0.075498,93.0,...,2.323946,2.753906,1.893986,0.859920,78.800563,77.677217,1.0,0.130920,0.063187,0.067733
349,2025-07-14,2781,2.835923,3.021792,2.827182,2.954977,1.239316e+10,1.747271e+11,0.531816,3.0,...,2.362154,2.870897,1.853412,1.017485,83.118311,79.490915,0.0,0.158256,0.082201,0.076055


### Preprocessing

We will only keep OHLCV data, market cap and the target feature from this set.

In [4]:
data.columns

Index(['date', 'name', 'open', 'high', 'low', 'close', 'volume', 'marketCap',
       'daily_weighted_sentiment', 'comment_volume', 'XRP_trends', 'gold_spot',
       'gspc_spot', 'ndx_spot', 'price_change', 'target', 'percent_change_24h',
       'percent_change_3d', 'percent_change_7d', 'percent_change_14d', 'ma_7d',
       'price_vs_ma7d', 'ma_14d', 'price_vs_ma14d', 'close_7_ema',
       'close_14_ema', 'dx', 'adx', 'adxr', 'rsi_7', 'rsi', 'stochrsi', 'atr',
       'mfi', 'boll', 'boll_ub', 'boll_lb', 'boll_width', 'kdjk', 'kdjd',
       'kdj_signal', 'macd', 'macds', 'macdh'],
      dtype='object')

In [5]:
df = data

In [6]:
df.head()

,date,name,open,high,low,close,volume,marketCap,daily_weighted_sentiment,comment_volume,...,boll,boll_ub,boll_lb,boll_width,kdjk,kdjd,kdj_signal,macd,macds,macdh
0,2024-07-30,2781,0.602084,0.634965,0.596157,0.627877,2.197506e+09,3.513567e+10,-0.075888,33.0,...,0.599856,0.633697,0.566015,0.067682,65.474038,61.924733,1.0,0.002721,0.001492,0.001229
1,2024-07-31,2781,0.627861,0.657911,0.623762,0.623939,2.419022e+09,3.491591e+10,-0.075888,33.0,...,0.601361,0.636201,0.566520,0.069681,61.604064,61.817844,-1.0,0.003579,0.001922,0.001657
2,2024-08-01,2781,0.623934,0.629433,0.571884,0.596464,1.990604e+09,3.340151e+10,0.001985,35.0,...,0.601073,0.634891,0.567255,0.067636,50.593588,58.076425,0.0,0.002489,0.002038,0.000451
3,2024-08-02,2781,0.596463,0.596466,0.549636,0.560771,1.884164e+09,3.140279e+10,0.000000,8.0,...,0.598834,0.636746,0.560922,0.075824,37.157042,51.103298,0.0,-0.000627,0.001495,-0.002122
4,2024-08-03,2781,0.560771,0.576546,0.542472,0.556391,1.325726e+09,3.115743e+10,0.000000,3.0,...,0.596600,0.638274,0.554926,0.083347,28.790651,43.665749,0.0,-0.003295,0.000523,-0.003818


In [7]:
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date', inplace=True)
df['day_of_week'] = df['date'].dt.dayofweek

In [8]:
df.head(10)

,date,name,open,high,low,close,volume,marketCap,daily_weighted_sentiment,comment_volume,...,boll_ub,boll_lb,boll_width,kdjk,kdjd,kdj_signal,macd,macds,macdh,day_of_week
0,2024-07-30,2781,0.602084,0.634965,0.596157,0.627877,2.197506e+09,3.513567e+10,-0.075888,33.0,...,0.633697,0.566015,0.067682,65.474038,61.924733,1.0,0.002721,0.001492,0.001229,1
1,2024-07-31,2781,0.627861,0.657911,0.623762,0.623939,2.419022e+09,3.491591e+10,-0.075888,33.0,...,0.636201,0.566520,0.069681,61.604064,61.817844,-1.0,0.003579,0.001922,0.001657,2
2,2024-08-01,2781,0.623934,0.629433,0.571884,0.596464,1.990604e+09,3.340151e+10,0.001985,35.0,...,0.634891,0.567255,0.067636,50.593588,58.076425,0.0,0.002489,0.002038,0.000451,3
3,2024-08-02,2781,0.596463,0.596466,0.549636,0.560771,1.884164e+09,3.140279e+10,0.000000,8.0,...,0.636746,0.560922,0.075824,37.157042,51.103298,0.0,-0.000627,0.001495,-0.002122,4
4,2024-08-03,2781,0.560771,0.576546,0.542472,0.556391,1.325726e+09,3.115743e+10,0.000000,3.0,...,0.638274,0.554926,0.083347,28.790651,43.665749,0.0,-0.003295,0.000523,-0.003818,5
5,2024-08-04,2781,0.556390,0.558306,0.516428,0.523396,1.209933e+09,2.930979e+10,-0.437253,8.0,...,0.645065,0.540814,0.104251,20.835438,36.055645,0.0,-0.007462,-0.001093,-0.006370,6
6,2024-08-05,2781,0.523396,0.525283,0.433492,0.489466,4.193894e+09,2.740930e+10,0.000000,1.0,...,0.658067,0.518822,0.139245,22.204183,31.438491,0.0,-0.012837,-0.003463,-0.009374,0
7,2024-08-06,2781,0.489458,0.518503,0.489296,0.505697,1.747525e+09,2.831866e+10,0.000000,1.0,...,0.658763,0.506047,0.152716,25.527500,29.468161,0.0,-0.015757,-0.005940,-0.009817,1
8,2024-08-07,2781,0.505691,0.640819,0.494230,0.600810,4.286386e+09,3.364709e+10,0.372538,3.0,...,0.660497,0.507436,0.153061,41.870342,33.602221,1.0,-0.011373,-0.007033,-0.004340,2
9,2024-08-08,2781,0.600849,0.641619,0.580908,0.617577,3.502680e+09,3.464894e+10,0.000000,5.0,...,0.663956,0.508359,0.155597,55.256088,40.820177,0.0,-0.006681,-0.006962,0.000282,3


In [9]:
df.columns

Index(['date', 'name', 'open', 'high', 'low', 'close', 'volume', 'marketCap',
       'daily_weighted_sentiment', 'comment_volume', 'XRP_trends', 'gold_spot',
       'gspc_spot', 'ndx_spot', 'price_change', 'target', 'percent_change_24h',
       'percent_change_3d', 'percent_change_7d', 'percent_change_14d', 'ma_7d',
       'price_vs_ma7d', 'ma_14d', 'price_vs_ma14d', 'close_7_ema',
       'close_14_ema', 'dx', 'adx', 'adxr', 'rsi_7', 'rsi', 'stochrsi', 'atr',
       'mfi', 'boll', 'boll_ub', 'boll_lb', 'boll_width', 'kdjk', 'kdjd',
       'kdj_signal', 'macd', 'macds', 'macdh', 'day_of_week'],
      dtype='object')

In [10]:
features = df[['day_of_week','name', 'open', 'high', 'low', 'close', 'volume', 'marketCap',
       'daily_weighted_sentiment', 'comment_volume', 'XRP_trends', 'gold_spot',
       'gspc_spot', 'ndx_spot', 'percent_change_24h',
       'percent_change_3d', 'percent_change_7d', 'percent_change_14d', 'ma_7d',
       'price_vs_ma7d', 'ma_14d', 'price_vs_ma14d', 'close_7_ema',
       'close_14_ema', 'dx', 'adx', 'adxr', 'rsi_7', 'rsi', 'stochrsi', 'atr',
       'mfi', 'boll', 'boll_ub', 'boll_lb', 'boll_width', 'kdjk', 'kdjd',
       'kdj_signal', 'macd', 'macds', 'macdh']]
target = df['target']
target = target.replace(-1, 0)

In [11]:
split_point = int(len(df) * 0.85)
X_test_set = features[split_point:] 
y_test_set = target[split_point:]   
actuals = y_test_set.tolist()

In [12]:
window_sizes = [int(len(df) * 0.85), 150, 100, 60, 30, 20]

Creating a sequence function for sequence models.

In [13]:
def create_sequences(features, target, time_steps=10):
    Xs, ys = [], []
    for i in range(len(features) - time_steps):
        Xs.append(features[i:(i + time_steps)])
        ys.append(target[i + time_steps])
    return np.array(Xs), np.array(ys)

TIME_STEPS = 10 

## Models

The models are tested against windows that include the following sizes: 
- 85% of the dataset (297 days)
- 150 days 
- 100 days
- 60 days
- 30 days
- 20 days

### Logistic Regression

In [14]:
for window_size in window_sizes:
    predictions = []
    
    for i in range(len(X_test_set)):
        end_index = split_point + i
        start_index = end_index - window_size
            
        X_train = features.iloc[start_index:end_index]
        y_train = target.iloc[start_index:end_index]

        # This takes one day's data at one time
        X_test = X_test_set.iloc[[i]] 

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model = LogisticRegression()
        model.fit(X_train_scaled, y_train)
        prediction = model.predict(X_test_scaled)[0]
        predictions.append(prediction)


    print(f"\nEvaluation for Logistic Regression with Window Size: {window_size}")
    current_actuals = actuals[len(actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for Logistic Regression with Window Size: 297
Accuracy: 0.6226
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.54      0.64      0.58        22
      UP (1)       0.70      0.61      0.66        31

    accuracy                           0.62        53
   macro avg       0.62      0.62      0.62        53
weighted avg       0.64      0.62      0.63        53

--------------------------------------------------

Evaluation for Logistic Regression with Window Size: 150
Accuracy: 0.5660
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.47      0.41      0.44        22
      UP (1)       0.62      0.68      0.65        31

    accuracy                           0.57        53
   macro avg       0.55      0.54      0.54        53
weighted avg       0.56      0.57      0.56        53

--------------------------------------------------

Evaluation for Logistic Regression with Window 

### Random Forest

In [15]:
for window_size in window_sizes:
    predictions = []
    
    for i in range(len(X_test_set)):
        end_index = split_point + i
        start_index = end_index - window_size
            
        X_train = features.iloc[start_index:end_index]
        y_train = target.iloc[start_index:end_index]
        X_test = X_test_set.iloc[[i]]

        model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train) 
        prediction = model.predict(X_test)[0]
        predictions.append(prediction)

    print(f"\nEvaluation for Random Forest with Window Size: {window_size}")
    current_actuals = actuals[len(actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for Random Forest with Window Size: 297
Accuracy: 0.6038
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.52      0.68      0.59        22
      UP (1)       0.71      0.55      0.62        31

    accuracy                           0.60        53
   macro avg       0.61      0.62      0.60        53
weighted avg       0.63      0.60      0.61        53

--------------------------------------------------

Evaluation for Random Forest with Window Size: 150
Accuracy: 0.5094
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.42      0.50      0.46        22
      UP (1)       0.59      0.52      0.55        31

    accuracy                           0.51        53
   macro avg       0.51      0.51      0.51        53
weighted avg       0.52      0.51      0.51        53

--------------------------------------------------

Evaluation for Random Forest with Window Size: 100
Accuracy

### XG Boost

In [16]:
for window_size in window_sizes:
    predictions = []
    
    for i in range(len(X_test_set)):
        end_index = split_point + i
        start_index = end_index - window_size
            
        X_train = features.iloc[start_index:end_index]
        y_train = target.iloc[start_index:end_index]
        X_test = X_test_set.iloc[[i]]

        model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
        model.fit(X_train, y_train) 
        prediction = model.predict(X_test)[0]
        predictions.append(prediction)

    print(f"\nEvaluation for XGB with Window Size: {window_size}")
    current_actuals = actuals[len(actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for XGB with Window Size: 297
Accuracy: 0.5472
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.47      0.64      0.54        22
      UP (1)       0.65      0.48      0.56        31

    accuracy                           0.55        53
   macro avg       0.56      0.56      0.55        53
weighted avg       0.58      0.55      0.55        53

--------------------------------------------------

Evaluation for XGB with Window Size: 150
Accuracy: 0.5283
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.44      0.55      0.49        22
      UP (1)       0.62      0.52      0.56        31

    accuracy                           0.53        53
   macro avg       0.53      0.53      0.53        53
weighted avg       0.54      0.53      0.53        53

--------------------------------------------------

Evaluation for XGB with Window Size: 100
Accuracy: 0.5660
Classification Report

### MLP Classifier

In [17]:
for window_size in window_sizes:
    predictions = []
    
    for i in range(len(X_test_set)):
        end_index = split_point + i
        start_index = end_index - window_size
            
        X_train = features.iloc[start_index:end_index]
        y_train = target.iloc[start_index:end_index]
        X_test = X_test_set.iloc[[i]]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model = Sequential([
            Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
            Dense(16, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train_scaled, y_train, epochs=10, batch_size=16, verbose=0)
    
        prediction = (model.predict(X_test_scaled, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)
        
    print(f"\nEvaluation for MLP with Window Size: {window_size}")
    current_actuals = actuals[len(actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for MLP with Window Size: 297
Accuracy: 0.4906
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.40      0.45      0.43        22
      UP (1)       0.57      0.52      0.54        31

    accuracy                           0.49        53
   macro avg       0.49      0.49      0.48        53
weighted avg       0.50      0.49      0.49        53

--------------------------------------------------

Evaluation for MLP with Window Size: 150
Accuracy: 0.5283
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.44      0.50      0.47        22
      UP (1)       0.61      0.55      0.58        31

    accuracy                           0.53        53
   macro avg       0.52      0.52      0.52        53
weighted avg       0.54      0.53      0.53        53

--------------------------------------------------

Evaluation for MLP with Window Size: 100
Accuracy: 0.5283
Classification Report

### LSTM

In [18]:
X_test_seq = pd.concat([features.iloc[split_point - TIME_STEPS:split_point], X_test_set])
y_test_seq = pd.concat([target.iloc[split_point - TIME_STEPS:split_point], y_test_set])

In [19]:
for window_size in window_sizes:
    predictions = []
    num_predictions_possible = len(X_test_seq) - TIME_STEPS
    loop_actuals = y_test_seq[TIME_STEPS:].tolist()

    for i in range(num_predictions_possible):
        end_index = split_point + i + TIME_STEPS
        start_index = end_index - window_size - TIME_STEPS

        window_features = features.iloc[start_index:end_index]
        window_target = target.iloc[start_index:end_index]

        scaler = StandardScaler()
        window_features_scaled = scaler.fit_transform(window_features)
        
        X_seq, y_seq = create_sequences(window_features_scaled, window_target.values, TIME_STEPS)
        X_train, y_train = X_seq[:-1], y_seq[:-1]
        X_test = X_seq[-1:]
        
        model = Sequential([
            LSTM(50, activation='relu', input_shape=(TIME_STEPS, X_train.shape[2])),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train, y_train, epochs=10, verbose=0)
        
        prediction = (model.predict(X_test, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)

    print(f"\nEvaluation for LSTM with Window Size: {window_size}")
    current_actuals = loop_actuals[len(loop_actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for LSTM with Window Size: 297
Accuracy: 0.6415
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.58      0.50      0.54        22
      UP (1)       0.68      0.74      0.71        31

    accuracy                           0.64        53
   macro avg       0.63      0.62      0.62        53
weighted avg       0.64      0.64      0.64        53

--------------------------------------------------

Evaluation for LSTM with Window Size: 150
Accuracy: 0.6415
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.59      0.45      0.51        22
      UP (1)       0.67      0.77      0.72        31

    accuracy                           0.64        53
   macro avg       0.63      0.61      0.61        53
weighted avg       0.63      0.64      0.63        53

--------------------------------------------------

Evaluation for LSTM with Window Size: 100
Accuracy: 0.6226
Classification Rep

### GRU

In [20]:
for window_size in window_sizes:
    predictions = []
    num_predictions_possible = len(X_test_seq) - TIME_STEPS
    loop_actuals = y_test_seq[TIME_STEPS:].tolist()

    for i in range(num_predictions_possible):
        end_index = split_point + i + TIME_STEPS
        start_index = end_index - window_size - TIME_STEPS

        window_features = features.iloc[start_index:end_index]
        window_target = target.iloc[start_index:end_index]

        scaler = StandardScaler()
        window_features_scaled = scaler.fit_transform(window_features)
        
        X_seq, y_seq = create_sequences(window_features_scaled, window_target.values, TIME_STEPS)
        X_train, y_train = X_seq[:-1], y_seq[:-1]
        X_test = X_seq[-1:]

        model = Sequential([
            GRU(50, activation='relu', input_shape=(TIME_STEPS, X_train.shape[2])),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train, y_train, epochs=10, verbose=0)
        
        prediction = (model.predict(X_test, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)

    print(f"\nEvaluation for GRU with Window Size: {window_size}")
    current_actuals = loop_actuals[len(loop_actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for GRU with Window Size: 297
Accuracy: 0.6038
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.53      0.45      0.49        22
      UP (1)       0.65      0.71      0.68        31

    accuracy                           0.60        53
   macro avg       0.59      0.58      0.58        53
weighted avg       0.60      0.60      0.60        53

--------------------------------------------------

Evaluation for GRU with Window Size: 150
Accuracy: 0.5849
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.50      0.41      0.45        22
      UP (1)       0.63      0.71      0.67        31

    accuracy                           0.58        53
   macro avg       0.56      0.56      0.56        53
weighted avg       0.58      0.58      0.58        53

--------------------------------------------------

Evaluation for GRU with Window Size: 100
Accuracy: 0.6038
Classification Report

### 1D-CNN

In [21]:
for window_size in window_sizes:
    predictions = []
    num_predictions_possible = len(X_test_seq) - TIME_STEPS
    loop_actuals = y_test_seq[TIME_STEPS:].tolist()

    for i in range(num_predictions_possible):
        end_index = split_point + i + TIME_STEPS
        start_index = end_index - window_size - TIME_STEPS

        window_features = features.iloc[start_index:end_index]
        window_target = target.iloc[start_index:end_index]

        scaler = StandardScaler()
        window_features_scaled = scaler.fit_transform(window_features)
        
        X_seq, y_seq = create_sequences(window_features_scaled, window_target.values, TIME_STEPS)
        X_train, y_train = X_seq[:-1], y_seq[:-1]
        X_test = X_seq[-1:]
        

        model = Sequential([
            Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(TIME_STEPS, X_train.shape[2])),
            MaxPooling1D(pool_size=2),
            Flatten(),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train, y_train, epochs=10, verbose=0)
        
        prediction = (model.predict(X_test, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)

    print(f"\nEvaluation for 1D-CNN with Window Size: {window_size}")
    current_actuals = loop_actuals[len(loop_actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for 1D-CNN with Window Size: 297
Accuracy: 0.7358
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.75      0.55      0.63        22
      UP (1)       0.73      0.87      0.79        31

    accuracy                           0.74        53
   macro avg       0.74      0.71      0.71        53
weighted avg       0.74      0.74      0.73        53

--------------------------------------------------

Evaluation for 1D-CNN with Window Size: 150
Accuracy: 0.5660
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.48      0.45      0.47        22
      UP (1)       0.62      0.65      0.63        31

    accuracy                           0.57        53
   macro avg       0.55      0.55      0.55        53
weighted avg       0.56      0.57      0.56        53

--------------------------------------------------

Evaluation for 1D-CNN with Window Size: 100
Accuracy: 0.6415
Classificati

### Ensemble: 1D-CNN and LSTM

In [22]:
for window_size in window_sizes:
    predictions = []
    num_predictions_possible = len(X_test_seq) - TIME_STEPS
    loop_actuals = y_test_seq[TIME_STEPS:].tolist()

    for i in range(num_predictions_possible):
        end_index = split_point + i + TIME_STEPS
        start_index = end_index - window_size - TIME_STEPS

        window_features = features.iloc[start_index:end_index]
        window_target = target.iloc[start_index:end_index]

        scaler = StandardScaler()
        window_features_scaled = scaler.fit_transform(window_features)
        
        X_seq, y_seq = create_sequences(window_features_scaled, window_target.values, TIME_STEPS)
        X_train, y_train = X_seq[:-1], y_seq[:-1]
        X_test = X_seq[-1:]
        

        model = Sequential([
            Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(TIME_STEPS, X_train.shape[2])),
            MaxPooling1D(pool_size=2),
            LSTM(50, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy')
        model.fit(X_train, y_train, epochs=10, verbose=0)
        
        prediction = (model.predict(X_test, verbose=0) > 0.5).astype("int32")[0][0]
        predictions.append(prediction)

    print(f"\nEvaluation for 1D-CNN-LSTM with Window Size: {window_size}")
    current_actuals = loop_actuals[len(loop_actuals) - len(predictions):]
    accuracy = accuracy_score(current_actuals, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(current_actuals, predictions, target_names=['DOWN (0)', 'UP (1)']))
    print("-" * 50)


Evaluation for 1D-CNN-LSTM with Window Size: 297
Accuracy: 0.5849
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.50      0.36      0.42        22
      UP (1)       0.62      0.74      0.68        31

    accuracy                           0.58        53
   macro avg       0.56      0.55      0.55        53
weighted avg       0.57      0.58      0.57        53

--------------------------------------------------

Evaluation for 1D-CNN-LSTM with Window Size: 150
Accuracy: 0.6038
Classification Report:
              precision    recall  f1-score   support

    DOWN (0)       0.53      0.45      0.49        22
      UP (1)       0.65      0.71      0.68        31

    accuracy                           0.60        53
   macro avg       0.59      0.58      0.58        53
weighted avg       0.60      0.60      0.60        53

--------------------------------------------------

Evaluation for 1D-CNN-LSTM with Window Size: 100
Accuracy: 0.52